This notebooks allows for the creation of sample ticker sets with category labels such as asset class, market capp, sector, etc.

In [38]:
import os
import pandas as pd
import random

Method 1: Trading View Screens Export

Step 1: Define parameters for building the sample ticker set

In [39]:
# Screen date for TradingView Screens data
screen_date = "02_24_25"
# What market caps should be included in the sample data
# market_caps = ["MICROCAP", "SMALLCAP", "MIDCAP", "LARGECAP", "MEGACAP"]
market_caps = ["MIDCAP", "LARGECAP", "MEGACAP"]

# How many samples for each market cap
n_samples_by_mktcap = 6

In [40]:
# Read all files in TradingView Screens folder with specified filters
# Define the folder path
folder_path = "TV_Screens"

# Create an empty list to store dataframes
dfs = []

# Loop through each market cap and read the corresponding files
for market_cap in market_caps:
    file_pattern = f"TV_SCREEN_US_STOCK_{market_cap}_{screen_date}.csv"
    file_path = os.path.join(os.getcwd(), folder_path, file_pattern)
    df = pd.read_csv(file_path)
    df['Market cap tag'] = market_cap
    dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

In [41]:
sample_ticker_list = []
# For each market cap
for market_cap in market_caps:
    # Filter the data
    temp_list = []
    filtered_df = combined_df[combined_df['Market cap tag'] == market_cap]
    # For each sector in the filtered data
    for sector in filtered_df['Sector'].unique():
        # Filter the data
        sector_df = filtered_df[filtered_df['Sector'] == sector]
        # Sample upto 2 tickers from the filtered data or all if less than 2
        for ticker in sector_df.sample(n=min(2, len(sector_df)))['Symbol']:
            temp_list.append({"ticker": ticker, "sector": sector, "market_cap": market_cap})

    sample_ticker_list.append(random.sample(temp_list, min(n_samples_by_mktcap, len(temp_list))))

flattened_ticker_list = [ticker for sublist in sample_ticker_list for ticker in sublist]

In [42]:
print(f"Number of tickers in sample = {len(flattened_ticker_list)}")

Number of tickers in sample = 18


Step 2: Define the parameters to fetch data for the sample ticker set

In [43]:
# Define the bar size for the sample data
bar_size = '1 hour'
# Define the duration and years_in_sample
duration = '1 Y' 
years_in_sample = 1 # Integer value - lowest value is 1

In [44]:
import nest_asyncio
nest_asyncio.apply()

import os
import sys

sys.path.append(os.path.join(os.getcwd(), "../.."))

from jarjarquant import Jarjarquant
jjq = Jarjarquant()

In [45]:
results = jjq.data_gatherer.get_random_price_samples_tws(years_in_sample=years_in_sample, tickers=[flattened_ticker_list[i]['ticker'] for i in range(len(flattened_ticker_list))], num_tickers_to_sample=len(flattened_ticker_list), bar_size=bar_size, duration=duration, verbose=True)

Error 162, reqId 7: Historical Market Data Service error message:HMDS query returned no data: IDA@SMART Trades, contract: Stock(symbol='IDA', exchange='SMART', currency='USD')


Error fetching data for IDA: 'NoneType' object has no attribute 'rename'


Error 162, reqId 10: Historical Market Data Service error message:No historical market data for 1514/STK@NYSENBBO Last 0, contract: Stock(symbol='LIN', exchange='SMART', currency='USD')


Error fetching data for LIN: unsupported operand type(s) for -: 'datetime.datetime' and 'list'


Error 162, reqId 11: Historical Market Data Service error message:Historical data request pacing violation, contract: Stock(symbol='LBTYA', exchange='SMART', currency='USD')


...

Error fetching data for J: 'NoneType' object has no attribute 'rename'


Error 162, reqId 25: Historical Market Data Service error message:HMDS query returned no data: OXY@SMART Trades, contract: Stock(symbol='OXY', exchange='SMART', currency='USD')


Error fetching data for OXY: 'NoneType' object has no attribute 'rename'


Error 162, reqId 33: Historical Market Data Service error message:HMDS query returned no data: HD@SMART Trades, contract: Stock(symbol='HD', exchange='SMART', currency='USD')


Error fetching data for HD: 'NoneType' object has no attribute 'rename'


In [49]:
flat_results = {key: value for result in results for key, value in result.items()}
ticker_data = {item['ticker']: {'market_cap': item['market_cap'], 'sector': item['sector']} for item in flattened_ticker_list}

sampled_ticker_data = {}
for k,v in flat_results.items():
    sampled_ticker_data[k] = ticker_data[k]

In [51]:
len(sampled_ticker_data)

10

Step 3: Name the sample and save sample data and metadata

In [47]:
from datetime import datetime
today_date = datetime.today().strftime('%Y-%m-%d')
timestamp = datetime.now().strftime('%H-%M-%S')

# Customize the folder name
sample_folder_name = f"1hour_midcap+_{today_date}_{timestamp}" 
os.makedirs(f"data/{sample_folder_name}", exist_ok=True)

In [48]:
# Pickle the ticker data
import pickle
with open(f"data/{sample_folder_name}/ticker_data_{bar_size}_{today_date}.pkl", "wb") as f:
    pickle.dump(sampled_ticker_data, f)

for key, value in flat_results.items():
    value.to_csv(f"data/{sample_folder_name}/{ticker_data[key]['market_cap']}_{ticker_data[key]['sector']}_{bar_size}_{today_date}.csv")